Import required modules

In [ ]:
import pydicom
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import json


Access the folder path for the cancer and the non-cancer images

In [ ]:
all_paths = json.loads(open("./paths.json").read())

personal_path = all_paths['personal_path']
non_cancerous_path = personal_path + all_paths['non_cancerous_path']
cancerous_path = personal_path + all_paths['cancerous_path']

Load in all the DICOM files

In [ ]:
def load_dicoms(folder_path):
    ct_dicom_list = []
    seg_dicom_list = []
    folder = os.listdir(folder_path)
    folder_size = len(folder)
    # go through each item in the folder
    for file_name in folder:
        # create a string with the path to the item being observed
        path = os.path.join(folder_path, file_name)
        # check if this is a ct file (folder will have multiple files and this file will be a dcm)
        if folder_size > 1 and file_name.endswith(".dcm"):
            ct_dicom_list.append(pydicom.dcmread(path))
        # check if this is an annotation file (folder should only have 1 file which will be a dcm)
        elif folder_size == 1 and file_name.endswith(".dcm"):
            seg_dicom_list.append(pydicom.dcmread(path))
        # check if this is folder, if so go into the folder
        elif os.path.isdir(path):
            ct_to_add, seg_to_add = load_dicoms(path)
            ct_dicom_list.extend(ct_to_add)
            seg_dicom_list.extend(seg_to_add)
    return ct_dicom_list, seg_dicom_list

def patient_dict(path):
    ct_dict = {}
    seg_dict = {}
    folder = os.listdir(path)
    for i in folder:
        cts, segs = load_dicoms(os.path.join(path, i))
        ct_dict[i] = cts
        seg_dict[i] = segs
    return ct_dict, seg_dict

non_cancerous_ct_dicom, non_cancerous_seg_dicom = patient_dict(non_cancerous_path)
cancerous_ct_dicom, cancerous_seg_dicom = patient_dict(cancerous_path)


DICOM to JPG

In [47]:
def load_dicom_images(dicoms):
    group = {}
    for i in dicoms:
        images = []
        for dicom in dicoms[i]:
            pix_array = dicom.pixel_array
            img_normalized = cv2.normalize(pix_array, None, 0, 255, cv2.NORM_MINMAX)
            img_uint8 = img_normalized.astype("uint8")
            images.append(img_uint8)
        group[i]=images
    return group

non_cancerous_ct_images = load_dicom_images(non_cancerous_ct_dicom)
non_cancerous_seg_images = load_dicom_images(non_cancerous_seg_dicom)
cancerous_ct_images = load_dicom_images(cancerous_ct_dicom)
cancerous_seg_images = load_dicom_images(cancerous_seg_dicom)

In [48]:

print("Non-cancerous images loaded:", len(non_cancerous_ct_images))
print("Annotated non-cancerous images loaded:", len(non_cancerous_seg_images))
print("Cancerous images loaded:", len(cancerous_ct_images))
print("Annotated cancerous images loaded:", len(cancerous_seg_images))

Non-cancerous images loaded: 3
Annotated non-cancerous images loaded: 3
Cancerous images loaded: 3
Annotated cancerous images loaded: 3


In [55]:
def convert_to_jpg(images, output_folder):
    for patient in images:
        new_folder = output_folder + "/" + patient
        if not os.path.exists(new_folder):
            os.makedirs(new_folder)
        for i, img in enumerate(images[patient]):
            cv2.imwrite(f"{new_folder}/image_{i}.jpg", img)


def convert_annotations_to_jpg(images, output_folder):
    for patient in images:
        new_folder = output_folder + "/" + patient
        if not os.path.exists(new_folder):
            os.makedirs(new_folder)
        for img in images[patient]:
            # since this is a 3d array, make a separate image for each layer in the 1st plane
            for i, j in enumerate(img):
                cv2.imwrite(f"{new_folder}/image_{i}.jpg", j)

cctip = personal_path + "/all_images/cancerous/ct"
csegip = personal_path + "/all_images/cancerous/segmentations"
ncctip = personal_path + "/all_images/non_cancerous/ct"
ncsegip = personal_path + "/all_images/non_cancerous/segmentations"

convert_to_jpg(cancerous_ct_images, cctip)
convert_to_jpg(non_cancerous_ct_images, ncctip)
convert_annotations_to_jpg(cancerous_seg_images, csegip)
convert_annotations_to_jpg(non_cancerous_seg_images, ncsegip)

Match CT and SEG Image Positions

In [13]:
def extract_ct_positions(ct_dicoms):
    all_ct_positions = {}
    for patient in ct_dicoms:
        ct_positions = {}
        # make a dictionary of ct image uid : ct image position
        for dicom in ct_dicoms[patient]:
            sop_instance_uid = dicom.SOPInstanceUID if hasattr(dicom, 'SOPInstanceUID') else None
            image_position = dicom.ImagePositionPatient if hasattr(dicom, 'ImagePositionPatient') else None
            if sop_instance_uid and image_position:
                ct_positions[sop_instance_uid] = image_position
        all_ct_positions[patient] = ct_positions
    return all_ct_positions

def extract_segmentation_positions(seg_dicoms):
    all_seg_positions = {}
    for patient in seg_dicoms:
        # This function assumes there is only 1 segmentation dicom/only the 1st segmentation dicom matters for each patient
        segmentation_positions = []
        # Locate the first DICOM file for this patient
        dicom = seg_dicoms[patient][0]  # Load the first segmentation file
        # make a list of (referenced ct image uid, seg image position)
        for i, frame in enumerate(dicom.PerFrameFunctionalGroupsSequence):
            sop_instance_uid = frame.DerivationImageSequence[0].SourceImageSequence[0].ReferencedSOPInstanceUID
            segmentation_positions.append((sop_instance_uid, i))
        all_seg_positions[patient] = segmentation_positions
    return all_seg_positions

def match_positions(ct_positions, segmentation_positions):
    all_matches = {}
    for patient in ct_positions:
        matched_positions = {}
        for (seg_uid, seg_frame) in segmentation_positions[patient]:
            if seg_uid in ct_positions[patient]:
                if not seg_uid in matched_positions:
                    matched_positions[seg_uid] = {
                        "CT_Position": ct_positions[patient][seg_uid],
                        "Segmentation_frames": [seg_frame]
                    }
                else:
                    matched_positions[seg_uid]["Segmentation_frames"].append(seg_frame)
        all_matches[patient] = matched_positions
    return all_matches

# Extract positions
try:
    non_cancerous_positions = extract_ct_positions(non_cancerous_ct_dicom)
    non_cancerous_annotation_positions = extract_segmentation_positions(non_cancerous_seg_dicom)
    cancerous_positions = extract_ct_positions(cancerous_ct_dicom)
    cancerous_annotation_positions = extract_segmentation_positions(cancerous_seg_dicom)
except ValueError as e:
    print(e)

# Match positions for non-cancerous and cancerous data
non_cancer_matched_positions = match_positions(non_cancerous_positions, non_cancerous_annotation_positions)
cancer_matched_positions = match_positions(cancerous_positions, cancerous_annotation_positions)

    
# Commented out print statements to reduce output

# # Check if data was extracted
# print("Non-cancerous CT Positions:", non_cancerous_positions)
# print("Non-cancerous Annotation Positions:", non_cancerous_annotation_positions)
# print("\nCancerous CT Positions:", cancerous_positions)
# print("Cancerous Annotation Positions:", cancerous_annotation_positions)

# # Check if there are matching SOP Instance UIDs for non-cancerous data
# print("\nNon-cancerous CT SOPInstanceUIDs:", list(non_cancerous_positions.keys()))
# print("Non-cancerous Segmentation SOPInstanceUIDs:", [uid for uid, _ in non_cancerous_annotation_positions])


# # Check if there are matching SOP Instance UIDs for cancerous data
# print("\nCancerous CT SOPInstanceUIDs:", list(cancerous_positions.keys()))
# print("Cancerous Segmentation SOPInstanceUIDs:", [uid for uid, _ in cancerous_annotation_positions])

# # Check matching data
# print(non_cancer_matched_positions)
# print(cancer_matched_positions)


{'ABD_LYMPH_001': {'61.7.225671413567587988709540514214837207016': {'CT_Position': [-195.5, -72.5, -3.716000e+02], 'Segmentation_frames': [0, 15, 31]}, '61.7.172515939084005252437456394015127410395': {'CT_Position': [-195.5, -72.5, -3.706000e+02], 'Segmentation_frames': [1, 16, 32]}, '61.7.311273956813205555334922523590492390726': {'CT_Position': [-195.5, -72.5, -3.696000e+02], 'Segmentation_frames': [2, 17, 33]}, '61.7.49239243940168297994111164026998337905': {'CT_Position': [-195.5, -72.5, -3.686000e+02], 'Segmentation_frames': [3, 18, 34]}, '61.7.46898277701274339226472024127606534162': {'CT_Position': [-195.5, -72.5, -3.676000e+02], 'Segmentation_frames': [4, 19, 35]}, '61.7.269707611950866927630519493863211594541': {'CT_Position': [-195.5, -72.5, -3.666000e+02], 'Segmentation_frames': [5, 20, 36]}, '61.7.62418700203484400719700631085943471490': {'CT_Position': [-195.5, -72.5, -3.656000e+02], 'Segmentation_frames': [6, 21, 37]}, '61.7.17033528246777316687879153860121593984': {'CT_P

Overlay annotations on CT images and produce a JPEG for visualization

In [11]:
def annotate(image, annotation, color):
    img = image
    for x, row in enumerate(annotation):
        for y, col in enumerate(row):
            # if there is an annotation
            if col > 0:
                img[y,x] = color
    return img

def overlay(ct_dicoms, ct_images, seg_images, matches, output_folder):
    # list of all images
    images = []
    match_found = 0
    # itterate through the dicoms
    for i, ct_dicom in enumerate(ct_dicoms):
        # check if this layer should have an annotation
        images.append(ct_images[i])
        if ct_dicom.SOPInstanceUID in matches:
            # this fumction assumes the ct and annotations are in the correct order
            img = images[i]
            # for cts that have multiple annotations for one slide, provide a different color for each slide 
            color_options = [[255,0,0],[0,255,0],[0,0,255]]
            # check that the image is in rgb format
            if len(img.shape) != 3:
                img = cv2.merge([img, img, img], None)
            # go through each frame in the seg file that corresponds to the ct frame and overlay the annotation
            for j, frame in enumerate(matches[ct_dicom.SOPInstanceUID]["Segmentation_frames"]):
                # pick a color to make each annotation different if there are multiple annotations in one frame
                color = color_options[j%len(color_options)]
                images[i] = annotate(img, seg_images[0][frame], color)
                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)
                cv2.imwrite(f"{output_folder}/image_{i}.jpg", images[i])
            match_found +=1
    # check that all matched frames have been annotated
    if match_found != len(matches):
        raise ValueError("there were " + len(matches) + "matches provided but only " + match_found + "matches found")
    return images

nc_output_folder = personal_path + "/all_images/non_cancerous/annotated_overlay"
c_output_folder = personal_path + "/all_images/cancerous/annotated_overlay"
non_cancerous_annotated_images = overlay(non_cancerous_ct_dicom, non_cancerous_ct_images, non_cancerous_seg_images, non_cancer_matched_positions, nc_output_folder)
cancerous_annotated_images = overlay(cancerous_ct_dicom, cancerous_ct_images, cancerous_seg_images, cancer_matched_positions, c_output_folder)

CNN

In [ ]:


X = non_cancerous_images_normalized + cancerous_images_normalized
y = [0] * len(non_cancerous_images_normalized) + [1] * len(cancerous_images_normalized)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Convert lists to arrays
X_train = np.array(X_train).reshape(-1, 224, 224, 1)  # Add channel dimension if grayscale
X_test = np.array(X_test).reshape(-1, 224, 224, 1)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (636, 224, 224, 1)
X_test shape: (159, 224, 224, 1)
y_train shape: (636,)
y_test shape: (159,)


In [ ]:
X_train

array([[[[-4.01568627],
         [-4.01568627],
         [-4.01568627],
         ...,
         [-4.01568627],
         [-4.01568627],
         [-4.01568627]],

        [[-4.01568627],
         [-4.01568627],
         [-4.01568627],
         ...,
         [-4.01568627],
         [-4.01568627],
         [-4.01568627]],

        [[-4.01568627],
         [-4.01568627],
         [-4.01568627],
         ...,
         [-4.01568627],
         [-4.01568627],
         [-4.01568627]],

        ...,

        [[-4.01568627],
         [-4.01568627],
         [-4.01568627],
         ...,
         [-4.01568627],
         [-4.01568627],
         [-4.01568627]],

        [[-4.01568627],
         [-4.01568627],
         [-4.01568627],
         ...,
         [-4.01568627],
         [-4.01568627],
         [-4.01568627]],

        [[-4.01568627],
         [-4.01568627],
         [-4.01568627],
         ...,
         [-4.01568627],
         [-4.01568627],
         [-4.01568627]]],


       [[[-4.01568627],


In [ ]:
X_test

array([[[[-7.84313725],
         [-7.84313725],
         [-7.84313725],
         ...,
         [-7.84313725],
         [-7.84313725],
         [-7.84313725]],

        [[-7.84313725],
         [-7.84313725],
         [-7.84313725],
         ...,
         [-7.84313725],
         [-7.84313725],
         [-7.84313725]],

        [[-7.84313725],
         [-7.84313725],
         [-7.84313725],
         ...,
         [-7.84313725],
         [-7.84313725],
         [-7.84313725]],

        ...,

        [[-7.84313725],
         [-7.84313725],
         [-7.84313725],
         ...,
         [-7.84313725],
         [-7.84313725],
         [-7.84313725]],

        [[-7.84313725],
         [-7.84313725],
         [-7.84313725],
         ...,
         [-7.84313725],
         [-7.84313725],
         [-7.84313725]],

        [[-7.84313725],
         [-7.84313725],
         [-7.84313725],
         ...,
         [-7.84313725],
         [-7.84313725],
         [-7.84313725]]],


       [[[-7.84313725],


In [ ]:
y_train

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,

In [ ]:
y_test

array([1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0])

In [ ]:
train_test_split(y, shuffle=False)

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


Check Class Distribution in Train and Test Sets

In [ ]:

# Assuming y_train and y_test are your labels for the train and test sets
train_class_distribution = pd.Series(y_train).value_counts(normalize=True)
test_class_distribution = pd.Series(y_test).value_counts(normalize=True)

print("Class distribution in training set:")
print(train_class_distribution)
print("\nClass distribution in testing set:")
print(test_class_distribution)

Class distribution in training set:
0    0.836478
1    0.163522
Name: proportion, dtype: float64

Class distribution in testing set:
0    0.811321
1    0.188679
Name: proportion, dtype: float64


Cross validation and bootstrapping